In [ ]:
#导入软件包
import numpy as np
import os
import sys
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets,svm,metrics,model_selection
from skimage import feature as ft

#图像处理部分
#读取数据
#取当前绝对路径加上相对路径
path = os.path.abspath(os.path.dirname(sys.argv[0]))+'\image'
categorys = os.listdir(path)
X = []
Y_label= []
#依次读取照片
for category in categorys:
    images = os.listdir(path+'/'+category)
    for image in images:
#基于hog的特征提取：计算像素点的方向梯度直方图特征
        im = ft.hog(Image.open(path+'/'+category+'/'+image).resize((256,256)),
                    orientations=9, 
                    pixels_per_cell=(8,8), 
                    cells_per_block=(8,8), 
                    block_norm = 'L2-Hys', #  block norm : str {‘L1’, ‘L1-sqrt’, ‘L2’, ‘L2-Hys’}
                    transform_sqrt = True, # power law compression 
                    feature_vector=True, # flatten the final vectors
                    visualise=False
                    )
        X.append(im)
        Y_label.append(category)
                
#将字符标签数字化
X = np.array(X)
Y_label = np.array(Y_label)
Y = LabelEncoder().fit_transform(Y_label)

#数据划分，其中训练数据占80%，测试数据占20%，同时将数据的顺序打乱
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.2, random_state=0)

#训练模型部分
  #各参数调节结果
    #kernel：核函数选择线性函数
    #C:正则化系数，C越大，易过拟合，C越小，易欠拟合
    #max_iter:最大迭代次数，max_iter越大越容易过拟合
 #调参技巧：对于线性函数来说，两个相互制约的参数为C和max_iter，将C调大的同时将max_iter调小，两者相互排列组合即可得到最优解
classifier = svm.SVC(kernel = 'linear',C =0.01,max_iter =8)
classifier.fit(x_train,y_train)
y_predict = classifier.predict(x_test)


#结果展示部分
#输出训练精度和测试精度
print(classifier.score(x_train,y_train),classifier.score(x_test,y_test))
#输出分类结果矩阵
print("Classification report for classifier %s:\n%s\n"
      %(classifier, metrics.classification_report(y_test, y_predict)))

